https://docs.google.com/document/d/118KdHMFEdTX6ghUDTG2Ev_qLDcZTM5HxKsT4JG6gLcA/edit
Main Task

You have to use the above dataset to build your recommender system.
Use of any other extra data is prohibited.
You can use pretrained models though and do some transfer learning.
Use NLP to build the recommender system. More points on how you use your NLP skills to build the recommender system.
You’ll have to use the overview of the movie and title compulsorily as they are related to your NLP transformations.
You can use other data given in the dataset as well, totally depends on you what you have to use and what not.
The output should give recommendations based on 1. Movie title 2. Movie overview 3. Movie genre.
You can think of more recommendations all by yourself. For eg. similar movies by year.

The optional bonus tasks you can do after you’ve completed the original task:

Bonus Task 1

Do topic modelling based on the text data available for movies.

Bonus Task 2

Deploy your models to get recommendation over HTTP APIs.


# Extracting contextual features using Bert

In [1]:
#Importing Dependencies

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
import bert
from bert.extract_features import *
import tensorflow_hub as hub
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [2]:
sess=tf.Session()

loss= -(y*log(ypred)+((1-y)*log(1-ypred)))

tfidf=tf * log(N/those doc having the word)

king-man+woman = queen

In [3]:
#simplifying data : extracting -> title , overview and genres

data=pd.read_csv("./movies_metadata.csv")

def make_dataset(data):
    data['tagline']=data["tagline"].fillna(" ")
    data=data.loc[:,["original_title","genres","overview"]]
    print(data.shape)
    data=data.dropna()  #dropping movies with no overview or genres
    print(data.shape)
#     data['description']=data.loc[:,"overview"]+data.loc[:,"tagline"]
    return data.loc[:,["original_title","genres"]],data.loc[:,"overview"]

data_info,data_view=make_dataset(data)

(45466, 3)
(44512, 3)


C:\Users\Dell\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
### Max len comes out to be 187 hence taking it 256 according to bert.

# y=data_view.apply(lambda x : x.split(" "))
# max_len=0
# for i in range(y.shape[0]):
#     if len(y.iloc[i])> max_len:
#         max_len=len(y.iloc[i])
# print(max_len)

max_len=256

In [5]:
#Using Tf hub to get details about the vocabulary etc of bert.

bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

max_seq_length = 64

def create_tokenizer_from_hub_module(bert_path,sess):     # gives bert tokenizer
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"],
        ]
    )

    return bert.tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

In [10]:
#extract features.py from google research/Bert Github 
def extract_emb(features,layers=[-1,-2,-3,-4],max_seq_len=64):  #extract features
    
    layer_indexes = [int(x) for x in layers]              #last 4 hidden layers

    bert_config = bert.modeling.BertConfig.from_json_file("./tmp/wwm_uncased_L-24_H-1024_A-16/bert_config.json")#model configs

    tokenizer = bert.tokenization.FullTokenizer(
      vocab_file="./tmp/wwm_uncased_L-24_H-1024_A-16/vocab.txt", do_lower_case=True)

    is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2    #
    run_config = tf.contrib.tpu.RunConfig(                          #
      master=None,                                                  #    Setting up configurations
      tpu_config=tf.contrib.tpu.TPUConfig(                          #  
          num_shards=None,                                          #
          per_host_input_for_training=is_per_host))                 #   

    unique_id_to_feature = {}
    for feature in features:
        unique_id_to_feature[feature.unique_id] = feature

    model_fn = model_fn_builder(                                        #
      bert_config=bert_config,                                          #
      init_checkpoint="./tmp/wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt",  #  funtion to build pretrained model
      layer_indexes=layer_indexes,                                      #
      use_tpu=False,                                                    # 
      use_one_hot_embeddings=False)                                     #
  
    estimator = tf.contrib.tpu.TPUEstimator(                            #
      use_tpu=False,                                                    #
      model_fn=model_fn,                                                # predictor to extract features 
      config=run_config,                                                #
      predict_batch_size=12)                                            #

    input_fn = input_fn_builder(features=features, seq_length=max_seq_len)   #prepares input
    
    trained_emb_data={} 
    i=0
    for result in estimator.predict(input_fn, yield_single_examples=True):
        l1=np.mean(axis=0,a=result['layer_output_0'])     #
        l2=np.mean(axis=0,a=result['layer_output_1'])     # Last 4 hidden layers features
        l3=np.mean(axis=0,a=result['layer_output_2'])     # 
        l4=np.mean(axis=0,a=result['layer_output_3'])     #

        emb=np.mean(axis=0,a=[l1,l2,l3,l4])               # taking mean to get sentence embeddings
        trained_emb_data[data_info.iloc[i,0]]=emb         # saving with movie titles
        i+=1
    return trained_emb_data                               # returning the features

In [33]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = data_view.apply(lambda x: InputExample(unique_id=0, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x, 
                                                                   text_b = None))
# Instantiate tokenizer
tokenizer = create_tokenizer_from_hub_module(bert_path,sess)

In [34]:
train_data=convert_examples_to_features(train_InputExamples
                                        ,max_len,tokenizer,) #converting to features expected by the model

In [14]:
# emb=extract_emb(train_data,max_seq_len=max_len) # extracting features

W0903 18:47:57.015343 13340 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x000001CE420D1D90>) includes params argument, but params are not passed to Estimator.
W0903 18:47:57.019330 13340 estimator.py:1811] Using temporary folder as model directory: C:\Users\Dell\AppData\Local\Temp\tmpkj539vy6
W0903 18:47:57.021613 13340 tpu_context.py:211] eval_on_tpu ignored because use_tpu is False.


In [22]:
# with open("./emb.pickle","wb") as file:   #saving embeddings
#     pkl.dump(emb,file)

In [2]:
import requests,bs4

In [4]:
html=requests.get("https://www.analyticsvidhya.com/blog/2017/05/41-questions-on-statisitics-data-scientists-analysts/")

In [6]:
html=bs4.BeautifulSoup(html.text)

In [7]:
html.find_all("strong")

[<strong>1) Which of these measures are used to analyze the central tendency of data? </strong>,
 <strong>Solution: (B)</strong>,
 <strong>Solution: (D)</strong>,
 <strong>Solution: (A)</strong>,
 <strong>Solution: (A)</strong>,
 <strong>5) Below, we have represented six data points on a scale where vertical lines on scale represent unit. </strong>,
 <strong>Which of the following line represents the mean of the given data points, where the scale is divided into same units?</strong>,
 <strong>Solution: (C)</strong>,
 <strong>6) If a positively skewed distribution has a median of 50, which of the following statement is true? </strong>,
 <strong>Solution: (E)</strong>,
 <strong>7) Which of the following is a possible value for the median of the below distribution?</strong>,
 <strong>Solution: (B)</strong>,
 <strong>8) Which of the following statements are true about Bessels Correction while calculating a sample standard deviation?</strong>,
 <strong>Bessels correction is always done when